In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/major2.csv')

In [ ]:
!pip install -q langchain_community
!pip install chromadb
!pip install pymysql
!pip install mysql-connector-python
!pip install faiss-gpu

In [ ]:
!nvidia-smi

Thu Nov 14 08:32:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              51W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [51]:
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
load_dotenv()


os.environ["RDS_ENDPOINT"]="bigcondb.cvyjxpa9co3t.ap-northeast-2.rds.amazonaws.com"
os.environ["RDS_USER"]="admin"
os.environ["RDS_REGION"]="ap-northeast-2b"
os.environ["RDS_NAME"]="tamdb"
os.environ["RDS_PW"]="xkafksmsakt"

# mysql 모듈 임포트
class MysqlClient:
    def __init__(self):
        self.endpoint = os.getenv("RDS_ENDPOINT")
        self.port = 3306
        self.user = os.getenv("RDS_USER")
        self.region = os.getenv("RDS_REGION")
        self.dbname = os.getenv('RDS_NAME')
        self.passwd = os.getenv('RDS_PW')
        os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'
        self.db_url = f"mysql+mysqlconnector://admin:{os.getenv('RDS_PW')}@{self.endpoint}:3306/{self.dbname}"
        self.engine = create_engine(self.db_url)
        self.conn = self.get_connection()
        self.cursor = self.conn.cursor()

    def get_connection(self):
        conn = pymysql.connect(host=self.endpoint, user=self.user, passwd=self.passwd, port=self.port,
                                     database=self.dbname)
        return conn


In [57]:
# mysql 객체 선언
mysql = MysqlClient()

query = f"select * from tamdb.detailed_info_new_test4"
mysql.cursor.execute(query)
rows = mysql.cursor.fetchall()

columns = [i[0] for i in mysql.cursor.description]  # 컬럼 이름 가져오기
df_basic = pd.DataFrame(rows, columns=columns)
df_basic= df_basic.fillna('')

In [ ]:
# len(df_basic[df_basic['score']==''])

3344

In [ ]:
df_basic_score = df_basic[df_basic['score']!=""]

In [ ]:
query = f"select * from tamdb.reviewsdb3"
mysql.cursor.execute(query)
rows = mysql.cursor.fetchall()

columns = [i[0] for i in mysql.cursor.description]  # 컬럼 이름 가져오기
df_review = pd.DataFrame(rows, columns=columns)
df_review= df_review.fillna('')

In [58]:
# v방문자 없는 경우 제외
df_full = df_basic[(df_basic['v_review_cnt']!='')&(df_basic['review']!='')]

In [59]:
df_full['ADDR_detail'] = df_full['ADDR'].map(lambda x: ' '.join(x.split(' ')[1:3]))

<ipython-input-59-8e28789a2c87>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['ADDR_detail'] = df_full['ADDR'].map(lambda x: ' '.join(x.split(' ')[1:3]))


In [ ]:
from tqdm.notebook import tqdm
from langchain.schema import Document
documents = []
for row in tqdm(df_full.iterrows()):
  document = Document(
      page_content=f"""<name>:{row[1]['MCT_NM']} </name>

      <review_summary>:{row[1]['review']}</review_summary>

      <full_location>:{row[1]['ADDR']}, </full_location>

      <average_score>:{row[1]['average_score']}, </average_score>

      <review_score>:{row[1]['score']}, </review_score>

      <average_price>:{row[1]['mean_price']},</average_price>

      <payment_method>:{row[1]['payment']}, </payment_method>

      <review_counts>: {row[1]['v_review_cnt']},</review_counts>

      <park_info>: {row[1]['park_info']},</park_info>

      <menus>: {row[1]['menus']}, </menus>

      <feature_info>:{row[1]['feature_tags']},</feature_info>

      <revisit_info>:{row[1]['revisit']},</revisit_info>

      <reservation_info>:{row[1]['reservation']},</reservation_info>

      <companion_info>:{row[1]['companion']},</companion_info>

      <waiting_info>:{row[1]['waiting_time']},</waiting_info>
      """,
      metadata={
        "name":row[1]['MCT_NM'],
                "review_summary":row[1]['review'],
                "full_location":row[1]['ADDR'],
                "location":row[1]['ADDR_detail'],
                "average_score":row[1]['average_score'],
                "review_score":row[1]['score'],
                "average_price":row[1]['mean_price'],
                "payment_method":row[1]['payment'],
                "review_counts": row[1]['v_review_cnt'],
                "park_info": row[1]['park_info'],
                'menus': row[1]['menus'],
                "menu_info":row[1]['menu_tags'],
                "feature_info":row[1]['feature_tags'],
                "revisit_info":row[1]['revisit'],
                "reservation_info":row[1]['reservation'],
                "companion_info":row[1]['companion'],
                "waiting_info":row[1]['waiting_time'],
                "type": row[1]['MCT_TYPE']
      }
  )
  documents.append(document)

0it [00:00, ?it/s]

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool
model_name = "upskyy/bge-m3-Korean"
model_kwargs = {
    "device": "cuda"
    # "device": "mps"
    # "device": "cpu"
}

encode_kwargs = {"normalize_embeddings": True}
hugging_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,)

<ipython-input-3-4f3e4f4c6d48>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hugging_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import Chroma
# hugging_vectorstore = Chroma.from_documents(
#     documents=documents,
#     embedding=hugging_embeddings,
#     persist_directory="/content/drive/MyDrive/competition/chroma_db"          # Save the embeddings at the first time
# )
